In [1]:
from SimCsstLens.LensPop.Population import LensPopulation
from SimCsstLens import CosmologyDistance as CD
import math
import h5py
import numpy as np 
from multiprocessing import Pool
import os
from astropy.table import Table, vstack

this_cosmos = CD.CosmosDist(Om0=0.3, Ode0=0.7, h=0.7)
lens_pop = LensPopulation(
    vdisp_floor=50, 
    zl_max=2.5, 
    cosmo_dist=this_cosmos,
    src_catalog_type='lsst',
    bands=['g', 'r', 'i', 'z'],
)
sky_frac = 17500.0/41252.96
N_etgs = lens_pop.dfl_pop.number_of_etgs(sky_frac=sky_frac) #ideal lenses
Nsamples_per_draw = 500000
Ndraw = math.ceil(N_etgs/Nsamples_per_draw)

In [3]:
current_dir = os.getcwd()
def return_detected(count_draw, stack=False):
    table = Table()

    fn = h5py.File(f"./samples/lenses_{count_draw}.hdf5", "r")
    src_thetaE = fn['source/thetaE'][()] #shape: [nsrc, n_ideal_lens]
    print(src_thetaE.shape, '111')
    for ii in range(src_thetaE.shape[0]):
        table[f'thetaE_s{ii}'] =  src_thetaE[ii, :]
        table[f'mass_s{ii}'] = fn['source/einstein_mass'][()][ii, :]
        table[f'z_s{ii}'] = fn['source']['z'][()][ii, :]
        table[f're_s{ii}'] = fn['source']['Re'][()][ii, :]
        table[f'q_s{ii}'] = fn['source']['q'][()][ii, :]
        table[f'pa_s{ii}'] = fn['source']['pa'][()][ii, :]
        table[f'x_s{ii}'] = fn['source']['xs'][()][ii, :]
        table[f'y_s{ii}'] = fn['source']['ys'][()][ii, :]
 
    table['vdisp_l'] = fn['deflector/vdisp'][()] #shape: [n_ideal_lens]
    table['re_l'] = fn['deflector/Re'][()]
    table['q_l']= fn['deflector/q'][()]
    table['z_l'] = fn['deflector/z'][()]
    table['mag_g_l'] = fn['deflector/app_mag_g'][()]
    table['mag_r_l'] = fn['deflector/app_mag_r'][()]
    table['mag_i_l'] = fn['deflector/app_mag_i'][()]
    table['mag_z_l'] = fn['deflector/app_mag_z'][()]

    src_bool = fn['source/bool_arr'][()]
    print(src_bool.shape, '222')
    print(np.all(src_bool), '222')
    fn.close()
    
    return table

table = return_detected(0, stack=True)

(2, 97) 111
(2, 97) 222
True 222


In [10]:
import numpy as np

a = np.arange(9).reshape(3, 3)
bool = np.array(
    [
        [True, True, True],
        [False, True, False],
        [True, False, True],
    ]
)

a[bool]

array([0, 1, 2, 4, 6, 8])

In [16]:
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [15]:
a.T[bool.T].reshape(-1, 2).T

array([[0, 1, 2],
       [6, 4, 8]])

In [12]:
a.T

array([[0, 3, 6],
       [1, 4, 7],
       [2, 5, 8]])

In [13]:
bool.T

array([[ True, False,  True],
       [ True,  True, False],
       [ True, False,  True]])